In [1]:
import  script

In [2]:
engine=script.create_conn()
cleaned_data=script.fetch_data(engine, "xdr_cleaned_data")

Connection successful


In [3]:
cleaned_data

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes),Start,End,Last Location Name,Handset Manufacturer,Handset Type
0,1.311448e+19,770.0000,662.00000,1.823652e+06,2.082014e+14,3.366496e+10,3.552121e+13,42.000000,5.000000,23.000000,...,1.434415e+07,1.717444e+08,8.814393e+06,3.674974e+07,3.088796e+08,4/4/2019 12:01,4/25/2019 14:35,9.16456699548519E+015,Samsung,Samsung Galaxy A5 Sm-A520F
1,1.311448e+19,235.0000,606.00000,1.365104e+06,2.082019e+14,3.368185e+10,3.579401e+13,65.000000,5.000000,16.000000,...,1.170709e+06,5.269042e+08,1.505514e+07,5.380039e+07,6.533850e+08,4/9/2019 13:04,4/25/2019 8:15,L77566A,Samsung,Samsung Galaxy J5 (Sm-J530)
2,1.311448e+19,1.0000,652.00000,1.361762e+06,2.082003e+14,3.376063e+10,3.528151e+13,109.795706,17.662883,6.000000,...,3.956300e+05,4.106926e+08,4.215763e+06,2.788364e+07,2.798073e+08,4/9/2019 17:42,4/25/2019 11:58,D42335A,Samsung,Samsung Galaxy A8 (2018)
3,1.311448e+19,486.0000,171.00000,1.321509e+06,2.082014e+14,3.375034e+10,3.535661e+13,109.795706,17.662883,44.000000,...,1.084972e+07,7.490399e+08,1.279728e+07,4.332422e+07,8.460285e+08,4/10/2019 0:31,4/25/2019 7:36,T21824A,undefined,undefined
4,1.311448e+19,565.0000,954.00000,1.089009e+06,2.082014e+14,3.369980e+10,3.540701e+13,109.795706,17.662883,6.000000,...,3.529801e+06,5.507095e+08,1.391032e+07,3.854281e+07,5.691386e+08,4/12/2019 20:10,4/25/2019 10:40,D88865A,Samsung,Samsung Sm-G390F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,7.277826e+18,451.0000,214.00000,8.123000e+04,2.082022e+14,3.365069e+10,3.548311e+13,32.000000,0.000000,52.000000,...,9.197207e+06,3.264510e+06,1.348742e+07,5.762885e+07,5.741753e+08,4/29/2019 7:28,4/30/2019 6:02,D20434A,Apple,Apple iPhone 8 Plus (A1897)
149997,7.349883e+18,483.0000,187.00000,9.797000e+04,2.082019e+14,3.366345e+10,3.566051e+13,27.000000,2.000000,23.000000,...,4.735033e+06,7.121804e+08,2.457758e+06,3.913508e+07,6.666488e+08,4/29/2019 7:28,4/30/2019 10:41,D10223C,Apple,Apple iPhone Se (A1723)
149998,1.311448e+19,283.0000,810.00000,9.824900e+04,2.082017e+14,3.362189e+10,3.572121e+13,43.000000,6.000000,43.000000,...,1.339432e+07,1.211009e+08,1.131473e+07,3.491222e+07,5.927864e+08,4/29/2019 7:28,4/30/2019 10:46,T51102A,Apple,Apple iPhone Xs (A2097)
149999,1.311448e+19,696.0000,327.00000,9.791000e+04,2.082021e+14,3.361962e+10,8.618620e+13,37.000000,5.000000,34.000000,...,2.529475e+06,8.147131e+08,1.406930e+06,2.962610e+07,3.718959e+08,4/29/2019 7:28,4/30/2019 10:40,L88342B,Huawei,Huawei Fig-Lx1


In [5]:
cleaned_data.columns

Index(['Bearer Id', 'Start ms', 'End ms', 'Dur. (ms)', 'IMSI', 'MSISDN/Number',
       'IMEI', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)',
       'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)',
       'TCP UL Retrans. Vol (Bytes)', 'DL TP < 50 Kbps (%)',
       '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)',
       'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)',
       '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)',
       'UL TP > 300 Kbps (%)', 'HTTP DL (Bytes)', 'HTTP UL (Bytes)',
       'Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Dur. (ms).1',
       'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',
       'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B',
       'Soci

In [21]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Assuming your dataframe is named df
# Extract relevant columns for engagement metrics
engagement_metrics = cleaned_data[['MSISDN/Number', 'Dur. (ms)', 'Total UL (Bytes)', 'Total DL (Bytes)']]

# Task 3.1: Aggregate the metrics per customer ID and report the top 10 customers
agg_engagement = engagement_metrics.groupby('MSISDN/Number').agg({
    'Dur. (ms)': 'sum',
    'Total UL (Bytes)': 'sum',
    'Total DL (Bytes)': 'sum'
}).reset_index()


agg_engagement['Total Engagement'] = agg_engagement['Dur. (ms)'] + agg_engagement['Total UL (Bytes)'] + agg_engagement['Total DL (Bytes)']


top_10_customers = agg_engagement.sort_values('Total Engagement', ascending=False).head(10)



In [22]:
top_10_customers

,MSISDN/Number,Dur. (ms),Total UL (Bytes),Total DL (Bytes),Total Engagement
106853,4.188282e+10,7.255100e+07,4.436642e+10,4.873783e+11,5.318173e+11
6437,3.361489e+10,9.966898e+06,6.894830e+08,8.156743e+09,8.856193e+09
92923,3.376054e+10,9.279434e+06,7.034786e+08,7.811295e+09,8.524053e+09
13180,3.362578e+10,1.855375e+07,7.295774e+08,7.770043e+09,8.518174e+09
13526,3.362632e+10,8.791927e+06,6.696507e+08,7.301517e+09,7.979959e+09
76363,3.367588e+10,4.865947e+06,5.815688e+08,7.309542e+09,7.895977e+09
37052,3.365973e+10,4.035428e+06,6.242603e+08,7.081602e+09,7.709898e+09
63028,3.366646e+10,4.536757e+06,4.050610e+08,6.903440e+09,7.313038e+09
92577,3.376041e+10,5.321667e+06,5.215189e+08,6.610852e+09,7.137692e+09
57241,3.366471e+10,2.927785e+06,4.712445e+08,6.400774e+09,6.874946e+09
